### 모델에 데이터가 들어가는 과정(번호가 순서는 아님)
1. 데이터를 모은다(collect, clean, preprocess)
2. Dataset Class로 전달
    - `__init__()`   : 초기 데이터를 어떻게 불러오는지 정의
        - file에서 불러오는지, 어떤 dir에서 불러오는지 등
    - `__len__()`    :데이터 길이
    - `__getitem__()`: __map-style__ 하나의 데이터를 불러올 때 어떻게 반환하는지 선언
3. transforms
    - 이미지 전처리, data augmentation 에서 data 변환 등의 전처리.
    - ToTensor() 를 거처 tensor형태로 데이터 받는다.
        - 데이터를 tensor로 바꾸는 작업은 transform(totensor)에서 일어남.
4. DataLoader
    - 데이터를 묶어서 모델에 넣어주는 역할
    - batch
    - shuffle
    - 등

### Dataset 클래스
- 데이터 입력 형태를 정의하는 클래스
- 데이터 입력하는 방식의 __표준화__. 모든 데이터에 적용됨
- image, text, audio 등 형태에 따른 다른 입력 정의

#### 유의점
- 데이터 형태에 따라 각 함수를 다르게 정의함
- 모든 것을 데이터 생성 시점에 처리할 필요 없음
    - image의 Tensor 변화는 학습에 필요한 시점에 변환 (transorm을 통해)
- 데이터 넷에 대한 표준화된 처리방법 제공 필요
    - 후속 연구자에게 빛
- Hugging Face 등 표준화된 라이브러리 사용



### 관련 모듈
- torch.utils.data
    - 데이터셋을 불러오고 자르고 섞는 도구 모음 모듈
- torch.utils.data.Dataset : 모델을 학습시키기 위한 데이터셋의 표준을 저장

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    # 초기 데이터를 어떻게 불러오는지 정의
    # file에서 불러오는지, 어떤 dir에서 불러오는지 등
    def __init__(self, text, labels):
            self.labels = labels
            self.data = text

    def __len__(self):
            return len(self.labels)

    # index 값을 주었을 때, 반환되는 데이터의 형태 (X,Y) 
    # 전처리하고 데이터 증강이 진행됨.
    def __getitem__(self, idx):
            label = self.labels[idx]
            text = self.data[idx]
            # 반환되는 형태
            sample = {"Text": text, "Class": label}
            return sample

In [4]:
text = ['Happy', 'Amazing', 'Sad', 'Unhapy', 'Glum']
labels = ['Positive', 'Positive', 'Negative', 'Negative', 'Negative']
MyDataset = CustomDataset(text, labels)

In [5]:
type(MyDataset)

__main__.CustomDataset

### DataLoader 클래스
- iterable 객체 생성
    - 객체를 iter()에 넣어주면 generater, 그 후 next()에 넣어 하나씩 호출가능
- Data의 Batch를 생성해주는 클래스
    -   Dataset은 하나의 데이터를 어떻게 가져올것인가.
    -   DataLoader는 index로 여러 데이터를 묶어서 모델에 넣어줌
- 학습 직전(GPU feed전) 데이터의 변환을 책임
- __Tensor로 변환__ + Batch 처리가 메인 업무
- 병렬적인 데이터 전처리 코드

In [11]:
# batch size = 2 --> 한번에 두개의 데이터씩 들어감
# shuffle = True --> text,lable을 한쌍으로 두개씩 랜덤으로 뽑아 넣는다.
MyDataLoader = DataLoader(MyDataset, batch_size=2, shuffle=True)
next(iter(MyDataLoader))
# 이제 gpu에 넣어줌

{'Text': ['Amazing', 'Happy'], 'Class': ['Positive', 'Positive']}

In [ ]:
MyDataLoader = DataLoader(MyDataset, batch_size=2, shuffle=True)
for dataset in MyDataLoader:
    print(dataset) 

{'Text': ['Amazing', 'Unhapy'], 'Class': ['Positive', 'Negative']}
{'Text': ['Sad', 'Glum'], 'Class': ['Negative', 'Negative']}
{'Text': ['Happy'], 'Class': ['Positive']}


하나의 EPIC을 돌리려면, 데이터로더가 한번 돌아가면된다.

#### DataLoader args

DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler = None, num_workers = 0, <br> &emsp;&emsp;&emsp;&emsp;&emsp; collate_fn = None, pin_memory=False, drop_last=False, timeout= 0, worker_init_fn= None,*,<br> &emsp;&emsp;&emsp;&emsp; &emsp;prefetch_factor =2, persistent_workers=False)
- collate_fn
    - [데이터, 레이블] 쌍으로 이루어진 형태를 , [데이터, 데이터,,], [레이블, 레이블,,] 형태로 바꾸어줌
    - 언제쓰이나요
        - text 데이터.
        - 길이가 각각 다른 text X에 0을 padding하여 동일한 길이로 만들때.
        

- batch_size
    - int, optional, default=1
    - 배치(batch)의 크기입니다. 데이터셋에 50개의 데이터가 있고, batch_size가 10라면 총 50/10=5, 즉 5번의 iteration만 지나면 모든 데이터를 볼 수 있습니다.
    - 이 경우 반복문을 돌리면 (batch_size, *(data.shape))의 형태의 Tensor로 데이터가 반환됩니다. dataset에서 return하는 모든 데이터는 Tensor로 변환되어 오니 Tensor로 변환이 안되는 데이터는 에러가 납니다.<br>
    <br>

- shuffle
    - bool, optional, default=False
    - 데이터를 DataLoader에서 섞어서 사용하겠는지를 설정할 수 있습니다. 실험 재현을 위해 torch.manual_seed를 고정하는 것도 포인트입니다.
    - 그냥 Dataset에서 initialize할 때, random.shuffle로 섞을 수도 있습니다.<br><br>


- sampler
    - Sampler, optional
    - torch.utils.data.Sampler 객체를 사용합니다.
    - sampler는 index를 컨트롤하는 방법입니다. 데이터의 index를 원하는 방식대로 조정합니다. 즉 index를 컨트롤하기 때문에 설정하고 싶다면 shuffle 파라미터는 False(기본값)여야 합니다.
    - map-style에서 컨트롤하기 위해 사용하며 __len__과 __iter__를 구현하면 됩니다. 그 외의 미리 선언된 Sampler는 다음과 같습니다.
        - SequentialSampler : 항상 같은 순서
        - RandomSampler : 랜덤, replacemetn 여부 선택 가능, 개수 선택 가능
        - SubsetRandomSampler : 랜덤 리스트, 위와 두 조건 불가능
        - WeigthRandomSampler : 가중치에 따른 확률
        - BatchSampler : batch단위로 sampling 가능
        - DistributedSampler : 분산처리 (torch.nn.parallel.DistributedDataParallel과 함께 사용)

https://subinium.github.io/pytorch-dataloader/ <br><br>

### 관련 모듈
- torchvision.dataset : torch.utils.data.Dataset을 상속하는 데이터셋 모음. MNIST, CIFAR-10 제공
- torchtext.dataset : 텍스트 데이터셋 모음. IMDb, AG_NEWS 제공
- torchvision.transforms : 여러 변환 필터를 담은 모듈. Tensor화, resize, crop, brightness 조절 가능
- torchvision.utils : 이미지데이터를 저장하고 시각화 하기 위한 모듈